In [3]:
import pandas as pd


def table_tonav(df):
    """Превращает таблицу, где первый столбец маги, а остальные нормы по тпг в таблицу для загрузки этих норм в нав. 
    По умолчанию максимальное значение 1, коэф отставания 0 индекс = номер магазина"""    
    
    norms_list = df.T.to_numpy().tolist()
    
    tpg_list = df.columns.tolist()
    
#добавим бк/иф к названиям тпг   
    new_tpg_list = ['БК_' + str(tpg) if int(str(tpg)[:4]) >= 1500 else 'ИФ_' + str(tpg) for tpg in tpg_list]
          
    tpg = [x for x in new_tpg_list for i in range(len(df.index.tolist()))]
    shop = df.index.tolist()*len(df.columns.tolist())
    norms = [round(item) for sublist in norms_list for item in sublist]#Все нормы округляются
    data = {'Товарная Группа':tpg,'Норма':norms,'Макс. допустимое кол-во на точке':[1]*len(norms),'коэф. Корр отставания проведения продаж':[0]*len(norms)}
    new_df = pd.DataFrame(index=shop, data=data)
    new_df.index.name = 'Код'                    
    return new_df

def priority_tonav(df):
    
#индекс = номер магазина    
    norms_list = df.T.to_numpy().tolist()
    tpg_list = df.columns.tolist()
    tpg = [x for x in tpg_list for i in range(len(df.index.tolist()))]
    shop = df.index.tolist()*len(df.columns.tolist())
    norms = [item for sublist in norms_list for item in sublist]#Все нормы округляются
    data = {'Товарное направление':tpg,'Приоритет':[100]*len(norms),'Повышающий коэффициент':[1]*len(norms),'Исключить из распределения':norms}
    new_df = pd.DataFrame(index=shop, data=data)
    new_df.index.name = 'Код'                    
    return new_df

In [2]:
# любая табла с нормами по тпг

table_tonav(df).to_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\в_нав.xlsx')

NameError: name 'df' is not defined

In [4]:
pd.read_excel(r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Татьяна ИФ.БК\РЕЙТИНГИ\топ100.xlsx', sheet_name='тпг_бк', index_col=1, skiprows=1)

,Unnamed: 0,0.93,БК КОЛЬЦА,БК КОЛЬЦА.1,БК КОЛЬЦА.2,БК КОЛЬЦА.3,БК КОЛЬЦА.4,БК КОЛЬЦА.5,БК КОЛЬЦА.6,БК КОЛЬЦА.7,...,БК ПОДВЕС ДЕКОР.8,БК ПОДВЕС ДЕКОР.9,БК ПОДВЕС ДЕКОР.10,БК ПОДВЕС ДЕКОР.11,БК ПОДВЕС ДЕКОР.12,БК ПОДВЕС ДЕКОР.13,БК ПОДВЕС ДЕКОР.14,БК ПОДВЕС ДЕКОР.15,БК ПОДВЕС ДЕКОР.16,БК ПОДВЕС ДЕКОР.17
0,NaN,NaN,1501.000000,1502.000000,1503.000000,1504.000000,1505.000000,1506.000000,1507.000000,1508.000000,...,1585.000000,1586.000000,1587.000000,1588.000000,1589.000000,1590.000000,1591.000000,1592.000000,1593.000000,1594.000000
1,1.0,5155.0,333.470455,317.406652,49.122893,390.576237,147.344534,57.079229,261.008732,305.929797,...,70.696945,180.572797,36.425951,8.031827,431.471667,575.295556,287.647778,36.718398,7.995928,3.226970
2,2.0,3052.0,257.248637,244.856560,37.894803,301.301669,113.665783,44.032548,200.487180,235.334794,...,50.762295,129.656091,26.154806,5.767067,303.216519,303.216519,202.144346,24.513646,5.338178,2.154364
3,3.0,4345.0,228.665455,217.650276,33.684270,267.823706,101.036252,39.140042,180.552885,211.001096,...,33.340060,85.156550,17.178160,3.787739,240.000000,120.000000,120.000000,15.318066,3.335716,1.346217
4,4.0,3816.0,238.193182,226.719037,35.087781,278.983027,105.246096,40.770878,184.218552,216.804142,...,38.485009,98.297679,19.829047,4.372253,240.097999,240.097999,120.049000,18.389186,4.004494,1.616120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,96.0,479.0,47.638636,45.343807,7.017556,55.796605,21.049219,8.154176,40.250000,46.000000,...,20.930642,53.460649,10.784321,2.377915,186.442843,93.221421,93.221421,10.712503,2.332793,0.941461
97,97.0,4338.0,95.277273,90.687615,14.035112,111.593211,42.098438,16.308351,80.500000,92.000000,...,1.603415,4.095413,0.826145,0.182163,8.337368,8.337368,8.337368,0.766215,0.166854,0.067338
98,98.0,4702.0,198.334787,188.780685,29.216317,232.299005,87.634591,33.948425,152.936519,180.171948,...,18.054988,103.382896,22.699142,5.872498,209.441228,186.169981,69.813743,31.230250,8.598415,2.793223
99,99.0,3197.0,87.655091,83.432606,12.912303,102.665754,38.730563,15.003683,74.060000,84.640000,...,24.728610,63.161346,12.741189,2.809399,241.343206,120.671603,120.671603,13.301824,2.896652,1.169021


In [2]:
#НОРМЫ ПО ТОП 100
bk_df = pd.read_excel(r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Татьяна ИФ.БК\РЕЙТИНГИ\топ100.xlsx', sheet_name='тпг_бк', index_col=1, skiprows=1)
if_df = pd.read_excel(r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Татьяна ИФ.БК\РЕЙТИНГИ\топ100.xlsx', sheet_name='тпг_иф', index_col=1, skiprows=1)

del bk_df['ранг']
del if_df['ранг']
#убираю значки '-'
bk_df = bk_df.apply(pd.to_numeric, errors='coerce').fillna(0)
if_df = if_df.apply(pd.to_numeric, errors='coerce').fillna(0)

result = table_tonav(bk_df).append(table_tonav(if_df))

result.to_excel(r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\в_нав.xlsx')

In [3]:
# ОБРУЧИ

#загружаем нормы по размерам и по тпг
wed_rings1 = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\нужное\матрица обручей.xlsx',
    sheet_name='гладкая_плоская размеры_тпг',index_col=0,skiprows=3)
del wed_rings1['адрес']
del wed_rings1['норма']

ring_cols = [0, 3, 8] + list(range(12,25))
wed_rings2 = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\нужное\матрица обручей.xlsx',
    sheet_name='Матрица по тпг',index_col=0,skiprows=3, usecols=ring_cols)

#применяем функцию исправляем максимумы соединяем в один датафрейм и сохраняем
wed_rings_tonav1 = table_tonav(wed_rings1)
wed_rings_tonav1['Макс. допустимое кол-во на точке'] = wed_rings_tonav1['Норма']
wed_rings_tonav2 = table_tonav(wed_rings2)

wed_rings_tonav = pd.concat([wed_rings_tonav1, wed_rings_tonav2])

wed_rings_tonav.to_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\в_нав.xlsx')

In [4]:
#МУСУЛЬМАНКА
from_26 = [0] + list(range(26,49))
mus_df = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\нужное\мусульманкэ.xlsx',
    sheet_name='нормы культ',index_col=0,skiprows=2,usecols=from_26)

mus_df.columns = [x[:4] for x in mus_df.columns.tolist()]

mus_df = table_tonav(mus_df)

mus_df.to_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\нужное\мус_внав.xlsx',
sheet_name='Лист1')

In [5]:
# ПО ТГ

#рассчитываем номера столбцов для каждого ТН

if_string_lenth = pd.read_excel(r'C:\Остатки\наполняемость с 10012020.xlsx', sheet_name='ИФ', nrows=0, skiprows=2)

lst = ['ИФ СЕРЬГИ' if 'ИФ СЕРЬГИ' in str(x) else x for x in if_string_lenth.columns.tolist()]

if_end = len(lst)-lst[::-1].index('ИФ СЕРЬГИ')
if_start = if_end-6
if_columnrange = [1,2] + list(range(if_start,if_end))

bk_string_lenth = pd.read_excel(r'C:\Остатки\наполняемость с 10012020.xlsx', sheet_name='БК', nrows=0, skiprows=2)

lst = ['БК СЕРЬГИ' if 'БК СЕРЬГИ' in str(x) else x for x in bk_string_lenth.columns.tolist()]

bk_end = len(lst)-lst[::-1].index('БК СЕРЬГИ')
bk_start = bk_end-6
bk_columnrange = [1,2] + list(range(bk_start,bk_end))


# загружаем датафреймы
begin = [1,2]

if_df = pd.read_excel(
    r'C:\Остатки\наполняемость с 10012020.xlsx', sheet_name='ИФ', skiprows=3, usecols=if_columnrange, index_col=1,
    names=['блок', 'СКЛАД', 'ИФ КОЛЬЦА', 'ИФ КОЛЬЦА ОБРУЧ', 'ИФ ПЕЧАТКИ', 'ИФ ПОДВЕС ДЕКОР', 'ИФ ПОДВЕС КУЛЬТ', 'ИФ СЕРЬГИ'])

bk_df = pd.read_excel(
    r'C:\Остатки\наполняемость с 10012020.xlsx', sheet_name='БК', skiprows=3, usecols=bk_columnrange, index_col=1,
    names=['блок', 'СКЛАД', 'БК КОЛЬЦА', 'БК КОЛЬЦА ОБРУЧ', 'БК ПЕЧАТКИ', 'БК ПОДВЕС ДЕКОР', 'БК ПОДВЕС КУЛЬТ', 'БК СЕРЬГИ'])

# убираем закрытые маги
if_df = if_df[~if_df.блок.str.contains('з', na=False)]
del if_df['блок']

bk_df = bk_df[~bk_df.блок.str.contains('з', na=False)]
del bk_df['блок']

#переворачиваем в нужный формат, соединяем и сохраняем
if_tonav = priority_tonav(if_df)
if_tonav.index.name = 'Код'

bk_tonav = priority_tonav(bk_df)
bk_tonav.index.name = 'Код'

total = if_tonav.append(bk_tonav)

total.to_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\приоритеты_внав.xlsx',
sheet_name='Лист1')


In [4]:
import pandas as pd
#пРИОРИТЕТЫ ОБРУЧИ В НАВ
path = 'C:/Остатки/болванка два.xlsx'
df = pd.read_excel(path, sheet_name='наполняха', skiprows=3,usecols=[1,2],names=['Код','Исключить из распределения'])
df['Товарное направление'], df['Приоритет'], df['Повышающий коэффициент'] = ['БК КОЛЬЦА ОБРУЧ', 100, 1]
df.set_index('Код', inplace=True)
order = ['Товарное направление', 'Приоритет', 'Повышающий коэффициент', 'Исключить из распределения']
df[order].to_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\приоритеты_внав.xlsx',
sheet_name='Лист1')

5